In [27]:
# SOLO CALCULO NUMERICO (POSIBLEMENTE NO SIRVE NADA)
import sympy as sp

# Defincion de simbolos
Avf2, wg = sp.symbols('Avf2 wg')

# Defincion de Constantes
Ad0 = 100 # Ganancia en dB
Avf = 20 # Ganancia en dB
w1 = 2 * sp.pi * 10 # Polo 1 del LM324
w2 = 2 * sp.pi * 5.06e6 # Polo 2 del LM324
fT = 1e6 # Frecuencia de cruce
wT = 2 * sp.pi * fT
M_fase = 65 * sp.pi / 180 # Margen de fase deseado

# Calculado manualmente:
Ri = 10e3
Rf = 90e3
R1 = 10e3
R2 = 36.6e3

# Margen de fase
eq1 = sp.Eq(sp.atan(wg / w1) + sp.atan(wg / wT), sp.pi - M_fase)

# Calculo de Margen de fase
wg = sp.nsolve(eq1, wg, (0, 1e3))
fg = wg / (2 * sp.pi)

# Pendiente de -40 dB/decada
eq2 = sp.Eq(Avf - Ad0 + 20 * (sp.log(wg/w1, 10)), Avf2)

# Calculo de Ganancia del lazo cerrado de AO2
Avf2 = sp.nsolve(eq2, Avf2, (0, 1e3))

# Calculo de Ganancia en banda media del amplificador compuesto
Ad0_compuesto = Ad0 + Avf2

# Conversion de dB a veces
Avf2_veces = 10 ** (Avf2 / 20)

# Calculo de Ancho de banda real
fH = fT/sp.sqrt((Ri+Rf)/Ri)

print(f"wg = {wg:.2f} rad/s")
print(f"fg = {fg:.2f} Hz")
print(f"Avf2 = {Avf2:.2f} dB = {Avf2_veces:.2f} veces")
print(f"Ad0_compuesto = {Ad0_compuesto:.2f} dB")
print(f"fH = {fH:.2f} Hz")

wg = 2930061.46 rad/s
fg = 466333.77 Hz
Avf2 = 13.37 dB = 4.66 veces
Ad0_compuesto = 113.37 dB
fH = 316227.77 Hz


In [7]:
# SOLO CALCULO NUMERICO 2 (POSIBLEMENTE NO SIRVE NADA)
import sympy as sp
import control as ct
import numpy as np
import matplotlib.pyplot as plt

# Defincion de simbolos
Avf2, wg = sp.symbols('Avf2 wg')

# Defincion de Constantes
Ad0_dB = 100 # Ganancia DC en dB
Ad0_veces = 100e3 # Ganancia DC en veces
Avf = 20 # Ganancia Lazo Cerrado en dB
f1 = 10 # Polo 1 del LM324
f2 = 5.06e6 # Polo 2 del LM324
w1 = 2 * sp.pi * f1 # Polo 1 del LM324
w2 = 2 * sp.pi * f2 # Polo 2 del LM324
fT = 1e6 # Frecuencia de cruce
wT = 2 * sp.pi * fT # Frecuencia angular de cruce
M_fase = 65 * sp.pi / 180 # Margen de fase deseado

# Valores calculados manualmente:
Ri = 10e3
Rf = 90e3
R1 = 10e3
R2 = 36.6e3

# Margen de fase
eq1 = sp.Eq(sp.atan(wg / w1) + sp.atan(wg / (2*sp.pi*631e3)), sp.pi - M_fase)

# Calculo de Margen de fase
wg = sp.nsolve(eq1, wg, (0, 1e3))
fg = wg / (2 * sp.pi)

# Pendiente de -40 dB/decada
eq2 = sp.Eq(Avf - Ad0_dB + 20 * (sp.log(wg/w1, 10)), Avf2)

# Calculo de Ganancia del lazo cerrado de AO2
Avf2 = sp.nsolve(eq2, Avf2, (0, 1e3))

# Calculo de Ganancia en banda media del amplificador compuesto
Ad0_dB_compuesto = Ad0_dB + Avf2

# Conversion de dB a veces
Avf2_veces = 10 ** (Avf2 / 20)

# Calculo de Ancho de banda potencial:
fH = fT/sp.sqrt((Ri+Rf)/Ri)

print(f"wg = {wg:.2f} rad/s")
print(f"fg = {fg:.2f} Hz")
print(f"Avf2 = {Avf2:.2f} dB = {Avf2_veces:.2f} veces")
print(f"Ad0_dB_compuesto = {Ad0_dB_compuesto:.2f} dB")
print(f"Ancho de banda potencial fH = {fH:.2f} Hz")


wg = 1848929.31 rad/s
fg = 294266.24 Hz
Avf2 = 9.37 dB = 2.94 veces
Ad0_dB_compuesto = 109.37 dB
Ancho de banda potencial fH = 316227.77 Hz


In [1]:
# Calculo de interseccion entre AV_LA y 1/K
# Graficos con libreria matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Defincion de Constantes
Ad0_dB = 100                # Ganancia DC en dB
Ad0_veces = 100e3           # Ganancia DC en veces
Avf = 20                    # Ganancia Lazo Cerrado en dB
f1 = 10                     # Polo 1 del LM324
f2 = 5.06e6                 # Polo 2 del LM324
w1 = 2 * np.pi * f1         # Polo 1 del LM324
w2 = 2 * np.pi * f2         # Polo 2 del LM324
fT = 1e6                    # Frecuencia de cruce
wT = 2 * np.pi * fT         # Frecuencia angular de cruce
M_fase = 65 * np.pi / 180   # Margen de fase deseado

# Valores calculados manualmente
Ri = 10e3
Rf = 90e3

# Funcion de transferencia Lazo abierto de LM324
num = [Ad0_veces * (2 * np.pi)**2 * f1 * f2]
den = [1, 2 * np.pi * (f1 + f2), (2 * np.pi)**2 * f1 * f2]

# Amplificador en Cascada
num_total = np.convolve(num, num)
den_total = np.convolve(den, den)
sistema_total = signal.TransferFunction(num_total, den_total)

# Cantidad de realimentacion K
K = Ri/(Ri+Rf)                  # Magnitud de K en veces
mag_1K_dB = 20 * np.log10(1/K)  # Magnitud de 1/K en dB

# Rango de frecuencias para grafico
f_min = 1                  # 1 Hz
f_max = 10e8               # 1 GHz
frecuencias = np.logspace(np.log10(f_min), np.log10(f_max), 1000)
w = 2 * np.pi * frecuencias

# Calculo de respuesta en frecuencia del sistema
w, mag, fase = signal.bode(sistema_total, w)

# Interseccion entre AV_LA_total y 1/K
frecuencias_cruce = np.logspace(3, 7, 10000)  # 1 kHz a 10 MHz
w_cruce = 2 * np.pi * frecuencias_cruce
w_cruce, mag_cruce, fase_cruce = signal.bode(sistema_total, w_cruce)

dif = np.abs(mag_cruce - mag_1K_dB)
idx = np.argmin(dif)
f_intersect = frecuencias_cruce[idx]
fase_intersect = fase_cruce[idx]

# Mostrar resultados numericos
print(f"Frecuencia de intersección: {f_intersect:.2f} Hz")
print(f"Magnitud: {mag_cruce[idx]:.2f} dB")
print(f"Fase: {fase_intersect:.2f}°")

# Graficos -------------------------------------------------------------------------------------------------
plt.figure(figsize=(7, 5), dpi=200)

# Modulo
plt.subplot(2, 1, 1)
plt.semilogx(frecuencias, mag, label='FT_LA_Total')
plt.semilogx(frecuencias, [mag_1K_dB]*len(frecuencias), label='1/K')  # Línea horizontal para 1/K
plt.title('Diagrama de Bode Amplificador VFA-VFA')
plt.ylabel('Magnitud [dB]')
plt.xlim(1e0, 1e9)  # Fuerza rango de 1 Hz a 1 GHz
plt.grid(which='both', linewidth=0.5)
plt.legend()

# Fase
plt.subplot(2, 1, 2)
plt.semilogx(frecuencias, fase)
plt.ylabel('Fase [°]')
plt.xlabel('Frecuencia [Hz]')
plt.xlim(1e0, 1e9)  # Fuerza rango de 1 Hz a 1 GHz
plt.ylim(-360, 0)  # Límites: mínimo=-360°, máximo=0°
plt.yticks(np.arange(-360, 1, 45))  # Pasos cada 45° desde -360 hasta 0
plt.grid(which='both', linewidth=0.5)

plt.tight_layout()
plt.show()

KeyboardInterrupt: 